# Trabalho Prático 2
**Grupo 22**

Alexis Correia - A102495 <br>
João Fonseca - A102512 <br>

## Enunciado
Considere o problema descrito no documento +Lógica Computacional: Multiplicação de Inteiros . Nesse documento usa-se um “Control Flow Automaton” como  modelo do programa imperativo que calcula a multiplicação de  inteiros positivos representados por vetores de bits.

  Pretende-se
  1. Construir um SFOTS, usando BitVec’s de tamanho $n$ , que descreva o comportamento deste autómato; para isso identifique e codifique em `Z3`  ou `pySMT`, as variáveis do modelo, o estado inicial , a relação de transição e o estado de erro.
  2. Usando $k$-indução verifique nesse SFOTS se $\,$ a propriedade $\,(x*y + z = a*b)\,$ $\,$ é um invariante do seu comportamento.
  3. Usando $k$-indução no FOTS acima e adicionando ao estado inicial  a condição  $\,$ $\,(a < 2^{n/2})\land(b < 2^{n/2})\,$, verifique a segurança do programa; nomeadamente  prove que, com tal estado inicial, o estado de erro nunca é acessível.

## Resolução

Um SFOTS é definido  por $\:\:\Sigma\;\equiv\;\langle\,\mathsf{X}\,,\,\mathsf{next}\,,\,\mathsf{I}\,,\,\mathsf{T}\,,\,\mathsf{E}\,\rangle\:\:$. Similiar a um FOTS, os estados são constituídos pelas variáveis do programa mais o __program counter__ (pc) e, tanto o estado inicial quanto as relações de trnsição, são caracterizados por predicados. A maior diferença se encontra na existência do estado de erro.

Comecemos, então, por analisar o programa imperativo em questão.

```Python
{a >= 0 and b >= 0}
0: x , y, z = a , b , 0
1: while not y == 0:
2:  if even(y):
3:      x , y , z = x << 1 , y >> 1 , z
4:  else:
5:      x , y , z = x , y - 1, z + x
6: stop
```
- Note-se que no final deve ser  $ z = a\times b $
- Neste pedaço de código, $ x $ e $ y $ são vetores de bits e as operações de ``<<``(Shift Left) e ``>>``(Shift Right) são equivalentes as operações ``*2`` e ``/2`` com inteiros.
<bl>
Podemos, ainda, supor que a função ``even`` se parece com algo do género:

```Python
0: def even(n):
1:   return (n%2==0)
```

- As variáveis do programa são: $ a $, $ b $, $ x $, $ y $ e $ z $
- O estado inicial é caracterizado pelo predicado: $\mathit{pc} = 0 \wedge a \ge 0 \wedge b \ge 0$
- As transições possíveis são caracterizadas das seguintes formas:
$$
\begin{array}{c}
(\mathit{pc} = 0 \wedge a \ge 0 \wedge b \ge 0 \wedge \mathit{pc}' = 1 \wedge a' = a \wedge b' = b \wedge x' = a \wedge y' = b \wedge z' = 0)\\
\vee\\
(\mathit{pc} = 1 \wedge y \neq 0 \wedge \mathit{pc}' = 2 \wedge a' = a \wedge b' = b \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
(\mathit{pc} = 1 \wedge y = 0 \wedge \mathit{pc}' = 6 \wedge a' = a \wedge b' = b \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
(\mathit{pc} = 2 \wedge (y\%2) = 0 \wedge \mathit{pc}' = 3 \wedge a' = a \wedge b' = b \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
(\mathit{pc} = 2 \wedge (y\%2) = 1 \wedge \mathit{pc}' = 5 \wedge a' = a \wedge b' = b \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
(\mathit{pc} = 3 \wedge \mathit{pc}' = 1 \wedge a' = a \wedge b' = b \wedge x' = x<<1 \wedge y' = y>>1 \wedge z' = z)\\
\vee\\
(\mathit{pc} = 5 \wedge \mathit{pc}' = 1 \wedge a' = a \wedge b' = b \wedge x' = x \wedge y' = y-1 \wedge z' = z+x)\\
\vee\\
(\mathit{pc} = 6 \wedge \mathit{pc}' = 6 \wedge a' = a \wedge b' = b \wedge x' = x \wedge y' = y \wedge z' = z)
\end{array}
$$
- O estado de erro acontece caso haja um ``overflow`` (tanto em $x = x << 1$  como em $z = z + x$);

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import BVType, INT

n = 16 # num de bits

# SFOTS #

def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i), INT)
    state['a'] = Symbol('a'+str(i), BVType(n))
    state['b'] = Symbol('b'+str(i), BVType(n))
    state['x'] = Symbol('x'+str(i), BVType(n))
    state['y'] = Symbol('y'+str(i), BVType(n))
    state['z'] = Symbol('z'+str(i), BVType(n))
    return state

def init(state):
    A = BVSGE(state['a'], BV(0,n)) 
    B = BVSGE(state['b'], BV(0,n))
    C = Equals(state['pc'], Int(0))
    return And(A,B,C)

def trans(curr, prox):
    t01 = And(Equals(curr['pc'], Int(0)), BVSGE(curr['a'], BV(0,n)), BVSGE(curr['b'], BV(0,n)),
              Equals(prox['pc'], Int(1)), Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']),
              Equals(prox['x'], curr['a']), Equals(prox['y'], curr['b']), Equals(prox['z'], BV(0,n)))
    
    t12 = And(Equals(curr['pc'], Int(1)), Not(Equals(curr['y'], BV(0,n))),
              Equals(prox['pc'], Int(2)), Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']),
              Equals(prox['x'], curr['x']), Equals(prox['y'], curr['y']), Equals(prox['z'], curr['z']))
    
    t16 = And(Equals(curr['pc'], Int(1)), Equals(curr['y'], BV(0,n)),
              Equals(prox['pc'], Int(6)), Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']),
              Equals(prox['x'], curr['x']), Equals(prox['y'], curr['y']), Equals(prox['z'], curr['z']))
    
    t23 = And(Equals(curr['pc'], Int(2)), Equals(BVExtract(curr['y'],0,0), BV(0,1)),
              Equals(prox['pc'], Int(3)), Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']),
              Equals(prox['x'], curr['x']), Equals(prox['y'], curr['y']), Equals(prox['z'], curr['z']))
    
    t25 = And(Equals(curr['pc'], Int(2)), Equals(BVExtract(curr['y'],0,0), BV(1,1)),
              Equals(prox['pc'], Int(3)), Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']),
              Equals(prox['x'], curr['x']), Equals(prox['y'], curr['y']), Equals(prox['z'], curr['z']))
    
    t31 = And(Equals(curr['pc'], Int(3)), Equals(prox['pc'], Int(1)), Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']),
              Equals(prox['x'], BVLShl(curr['x'], 1)), Equals(prox['y'], BVLShr(curr['y'], 1)), Equals(prox['z'], curr['z']))
    
    t51 = And(Equals(curr['pc'], Int(5)), Equals(prox['pc'], Int(1)), Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']),
              Equals(prox['x'], curr['x']), Equals(prox['y'], BVSub(curr['y'], BV(1,n))), Equals(prox['z'], BVAdd(curr['z'],curr['x'])))
    
    t66 = And(Equals(curr['pc'], Int(6)), Equals(prox['pc'], Int(6)), Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']),
              Equals(prox['x'], curr['x']), Equals(prox['y'], curr['y']), Equals(prox['z'], curr['z']))
    
    return Or(t01, t12, t16, t23, t25, t31, t51, t66)

def error(state):
    of_Shl = BVULT(BV(2**n - 1, n), BVLShl(state['x'], 1))
    of_Add = BVULT(BV(2**n - 1, n), BVAdd(state['z'], state['x']))
    return Or(of_Shl, of_Add)


Dessa forma, está pronta nosso **SFOTS**. Com as funções acima (``declare``, ``init``, ``trans`` e ``error``) devidamente definidas, podemos escrever ``genTrace`` que escreverá o traço de comprimento $K$ e o valor de cada variável em cada estado $i$ tal que $i\in {0,1,2,...,K}$.

In [ ]:
def genTrace(declare, init, trans, error, K):
    with Solver(name="z3") as s:
        X = [declare(i) for i in range(K+1)]
        I = init(X[0])
        Tks = [trans(X[i],X[i+1]) for i in range(K) ]
        
        if s.solve([I,And(Tks) ]):
            for i in range(K):
                print("Estado:",i)
                print(f"          pc = {s.get_value(X[i]['pc'])}")
                print(f"          a = {s.get_value(X[i]['a'])}, b = {s.get_value(X[i]['b'])}")
                for v in ['x', 'y', 'z']:
                    p = format(s.get_value(X[i][v]).constant_value(), f'0{n}b')
                    print(f'          {v} = {p} ({s.get_value(X[i][v])})')
        else:
            print("ERROR")

genTrace(declare, init, trans, error, 20)

Agora, verificaremos se a propriedade $\,(x*y + z = a*b)\,$ $\,$ é um invariante do comportamento desta SFOTS com $k$-indução.

Primeiramente, podemos afirmar empiricamente que a variável $y$ sempre chega ao final do código com valor 0 e, por isso, $(x*y + z = a*b)\implies (z=a*b) $ que é da facto o que este código calcula. Logo, sabemos que no fim da execução, esta equação é verdadeira (Desde que não haja ``overflow``).

Porém, ainda precisamos verificar com auxílio de $k$-indução se a afirmação é verdadeira em todos os estados e se esta propriedade é de facto invariante de SFOTS.

In [ ]:
def kinduction_always(declare,init,trans,inv,k):
    with Solver(name="z3") as solver:
        s = [declare(i) for i in range(k)]
        solver.add_assertion(init(s[0]))
        for i in range(k-1):
            solver.add_assertion(trans(s[i],s[i+1]))
            
        for i in range(k):
            solver.push()
            solver.add_assertion(Not(inv(s[i])))
            if solver.solve():
                print(f"> Contradição! O invariante não se verifica nos k estados iniciais.")
                return
            solver.pop()
        
        s2 = [declare(i+k) for i in range(k+1)]
        
        for i in range(k):
            solver.add_assertion(inv(s2[i]))
            solver.add_assertion(trans(s2[i],s2[i+1]))
        
        solver.add_assertion(Not(inv(s2[-1])))
        
        if solver.solve():
            print(f"> Contradição! O passo indutivo não se verifica.")
            return
        
        print(f"> A propriedade verifica-se por k-indução (k={k}).")
    return
 
def inv(state): # Invariante: x*y+z=a*b
    R = BVAdd(BVMul(state['x'],state['y']),state['z'])
    L = BVMul(state['a'], state['b'])
    return Equals(R, L)

kinduction_always(declare,init,trans,inv, 2)

Com as funções ``kinduction_always`` e ``inv``, pudemos verificar que, de facto, a propriedade não invariante. Ela falha logo no primeiro estado (estado inicial).

Por fim, vamos averiguar a segurança do programa ao adicionar a condição $(a < 2^{n/2})\land(b < 2^{n/2})$ ao estado inicial. Mais uma vez, nos utilizaremos da $k$-indução neste exercício.

In [ ]:
def init1(state):
    I = init(state)
    maxA = BVSLT(state['a'], BV(2**(n//2),n))
    maxB = BVSLT(state['b'], BV(2**(n//2),n))
    return And(I,maxA,maxB)

def safe(state):
    X = BVULE(state['x'], BV((2**n)-1,n))
    Z = BVULE(state['z'], BV((2**n)-1,n))
    return  And(X,Z)

kinduction_always(declare, init1, trans, safe, 20)

Como podemos ver, ao adicionar esta nova condição ao estado inicial, evitamos por completo que aconteça **overflow** do $x$ e do $z$.